In [274]:
import pandas as pd
import numpy as np

# Contribución empírica del crecimiento económico: Estimaciones OLS, CLS y EMD

1. Carga la base de datos en un DataFrame.

In [275]:
url=r'C:\Users\Camil\Documents\GitHub\ECOP2037_CE\hmw4\MRW1992.dta'
df=pd.read_stata(url)

2. Calcula las matrices X (con columna de 1s) y y a partir del DataFrame.

In [276]:
df = df[df['N'] != 0]

log_change=np.log(df['Y85'])-np.log(df['Y60'])

logGDP60=np.log(df['Y60'])

logI_GDP=np.log(df['invest']/100)

log_school=np.log(df['school']/100)

log_growth=np.log(df['pop_growth']/100 +0.05)

In [277]:
y=np.array(log_change)

x=np.column_stack([np.ones(len(df)), logGDP60, logI_GDP, log_growth,log_school])

3. Implementa el estimador OLS y sus su matriz de varianza y covarianza HCO.

In [278]:
beta_ols= (np.linalg.inv(x.T @ x)) @ (x.T @ y)
residuos = y - x @ beta_ols
n, k = x.shape
m = np.zeros((k, k))
for i in range(n):
    xi = x[i, :].reshape(-1, 1)
    ui2 = residuos[i]**2         
    m += ui2 * (xi @ xi.T)          

var_covOLS=(n/(n-k)) *(np.linalg.inv(x.T @ x)) @ m @ np.linalg.inv(x.T @ x)
se=np.sqrt(np.diag(var_covOLS))


4. Implementa el estimador CLS y su matriz de varianza y covarianza.

In [279]:
R = np.array([[0], [0], [1], [1], [1]])
r = np.array([0])

beta_cls= beta_ols- ((np.linalg.inv(x.T @x)) @ R @ np.linalg.inv(R.T @ np.linalg.inv(x.T @x) @ R) @ ((R.T@beta_ols) -r ))

residuosCLS = (y-x@beta_cls).reshape(-1, 1)
n, k = x.shape
q = R.shape[0]
omega = (residuosCLS.T @ residuosCLS) / (n - k + q)
correction = (np.linalg.inv(x.T @x)) @ R @ (np.linalg.inv(R.T @ (np.linalg.inv(x.T @x)) @ R)) @ R.T @ (np.linalg.inv(x.T @x))
var_covCLS = omega.item() * ((np.linalg.inv(x.T @x)) - correction)


seCLS=np.sqrt(np.diag(var_covCLS))

5. Implementa el estimador EMD. Pondera por una matriz de varianza estimada eficiente bV ,
siguiendo el mismo principio de CLS. Estima también la matriz de varianza y covarianza.

In [280]:
XtX_inv = np.linalg.inv(x.T @ x)

beta_emd = beta_ols - var_covOLS @ R @ np.linalg.inv((R.T @ var_covOLS @ R)) @ (R.T @ beta_ols - r)

y_hat_emd = x @ beta_emd
residuosEMD= y - y_hat_emd

Omega_hc0_emd = np.diag(residuosEMD**2)

var_covEMD= var_covOLS- (var_covOLS@ R @ (np.linalg.inv(R.T @ var_covOLS @ R)) @R.T @ var_covOLS)
se_emd = np.sqrt(np.diag(var_covEMD))

6. Implementa un código que genere una tabla de resumen de resultado como la Tabla 8.1

In [281]:
columna1= [ "Intercept","logGDP1960", "logI/GDP", 'log(n+g+d)', "log(school)"]

tabla = pd.DataFrame({
    'Variable': columna1,
    'OLS': beta_ols.flatten(),
    'SE_OLS': se.flatten(),
    'CLS': beta_cls.flatten(),
    'SE_CLS': seCLS.flatten(),
    'EMD': beta_emd.flatten(),
    'SE_EMD': se_emd.flatten()
})
tabla = tabla.round(2)
tabla

,Variable,OLS,SE_OLS,CLS,SE_CLS,EMD,SE_EMD
0,Intercept,3.02,0.74,2.46,0.46,2.48,0.44
1,logGDP1960,-0.29,0.05,-0.30,0.06,-0.30,0.05
2,logI/GDP,0.52,0.11,0.50,0.08,0.46,0.08
3,log(n+g+d),-0.51,0.24,-0.74,0.08,-0.71,0.07
4,log(school),0.23,0.07,0.24,0.06,0.25,0.06


7. Escriba un párrafo breve explicando las diferencias entre OLS, CLS y EMD en términos
econométricos, y sobretodo su interpretación ecónomica.

El estimador OLS puede ser ineficiente en ciertos casos en los que existen restricciones derivadas de la teoria que deben ser incorporadas como el modelo de solow. Usando el estimador CLS se imponen estas restricciones lineales a los coeficientes lo que puede dar mejores estimaciones o mas apegadas a la realidad si las restricciones son pertinentes. Sin embargo, este estimador no toma en cuenta la estructura de varianza y covarianza de los errores. El estimador EMD ajusta esas restricciones usando una matriz de ponderación eficiente derivada de la varianza estimada. En términos económicos, OLS estima relaciones ignorando conceptos teoricos, mientras, CLS y EMD introducen estos conceptos y lo aprovechan para obtener estimaciones mas eficientes en terminos economicos, siendo EMD superior si las restricciones son válidas y hay heterocedasticidad.